<a href="https://colab.research.google.com/github/niks1503/TripSync-AI-Trip-Planner/blob/main/EDA_Model_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df1=pd.read_csv("/content/drive/MyDrive/Maharashtra datasets/Tourism_Footfall.csv")
df1.head()

,State,District,Place_Name,Year,Domestic_Visitors,Foreign_Visitors,Total_Visitors
0,Maharashtra,Ahmednagar,District_total,2020,5464844,10574,5475418
1,Maharashtra,Mumbai city,District_total,2020,3100519,426286,3526805
2,Maharashtra,Mumbai suburban,District_total,2020,2397055,115092,2512147
3,Maharashtra,Pune,District_total,2020,4780726,114914,4895640
4,Maharashtra,Aurangabad,District_total,2020,1989244,223727,2212971


In [4]:
df2=pd.read_csv("/content/drive/MyDrive/Maharashtra datasets/Travel_cost.csv")
df2.head()

,Place_Name,District,Category,Item,Cost_INR,Audience,Notes
0,Karnala bird sanctuary,Raigad,Entry fee,Entry ticket,50.0,Per person,Sanctuary entry
1,Futala lake,Nagpur,Entry fee,Public access,0.0,All,Open to public / free entry
2,Futala lake,Nagpur,Activity,Boating (paddle/row),200.0,Per person,Boating estimate
3,Pench national park,Nagpur,Activity,Safari (gypsy full),6500.0,Group (6),Permit + vehicle + guide
4,Pench national park,Nagpur,Entry fee,Park entry,100.0,Per person,Gate entry


In [5]:
df3=pd.read_csv("/content/drive/MyDrive/Maharashtra datasets/Weather_dataset.csv")
df3.head()

,State,District,Month,Avg_Temperature_C,Rainfall_mm,Humidity,Weather_Type
0,Maharashtra,Ahmednagar,January,22.2,0.0,30,Dry
1,Maharashtra,Ahmednagar,February,21.6,0.0,37,Dry
2,Maharashtra,Ahmednagar,March,33.4,9.0,21,Hot
3,Maharashtra,Ahmednagar,April,32.9,13.0,22,Hot
4,Maharashtra,Ahmednagar,May,32.9,0.0,21,Hot


In [6]:
df4=pd.read_csv("/content/drive/MyDrive/Maharashtra datasets/Travel_distance.csv")
df4.head()

,Source_Place_ID,Destination_Place_ID,Distance_KM,Travel_Time_Min,Transport_Mode
0,Mh_00001,Mh_07443,2.20,3,Car/taxi
1,Mh_00001,Mh_00331,2.27,3,Car/taxi
2,Mh_00001,Mh_00158,2.45,3,Car/taxi
3,Mh_00001,Mh_03196,6.93,10,Car/taxi
4,Mh_00001,Mh_00834,11.76,17,Car/taxi


In [7]:
df_costs = df2.groupby(['Place_Name', 'District'])['Cost_INR'].mean().reset_index()
df_costs.head()

,Place_Name,District,Cost_INR
0,105-feet hanuman statue,Buldhana,0.0
1,360 degree adventure,Pune,0.0
2,8 to 80 park,Pune,0.0
3,A c depo musjid,Ahmednagar,0.0
4,Aaambhira buddha vihar,Ratnagiri,0.0


In [8]:
df_weather = df3.groupby('District')[['Avg_Temperature_C', 'Rainfall_mm', 'Humidity']].mean().reset_index()
df_weather.head()

,District,Avg_Temperature_C,Rainfall_mm,Humidity
0,Ahmednagar,27.141667,66.916667,44.416667
1,Akola,26.816667,79.666667,49.333333
2,Amravati,27.075000,77.166667,48.583333
3,Aurangabad,27.025000,62.833333,45.166667
4,Beed,26.908333,65.416667,43.916667


In [9]:
merged_df = pd.merge(df1, df_costs, on=['Place_Name', 'District'], how='left')
merged_df = pd.merge(merged_df, df_weather, on='District', how='left')
merged_df.head()

,State,District,Place_Name,Year,Domestic_Visitors,Foreign_Visitors,Total_Visitors,Cost_INR,Avg_Temperature_C,Rainfall_mm,Humidity
0,Maharashtra,Ahmednagar,District_total,2020,5464844,10574,5475418,NaN,27.141667,66.916667,44.416667
1,Maharashtra,Mumbai city,District_total,2020,3100519,426286,3526805,NaN,26.866667,227.666667,74.833333
2,Maharashtra,Mumbai suburban,District_total,2020,2397055,115092,2512147,NaN,26.491667,229.166667,75.000000
3,Maharashtra,Pune,District_total,2020,4780726,114914,4895640,NaN,24.116667,58.333333,52.500000
4,Maharashtra,Aurangabad,District_total,2020,1989244,223727,2212971,NaN,27.025000,62.833333,45.166667


In [10]:
print(f"Number of rows with missing Cost_INR after merge: {merged_df['Cost_INR'].isnull().sum()}")

Number of rows with missing Cost_INR after merge: 195


In [11]:
merged_df['Cost_INR'].fillna(merged_df['Cost_INR'].mean(), inplace=True)
print(f"Number of rows with missing Cost_INR after imputation: {merged_df['Cost_INR'].isnull().sum()}")

Number of rows with missing Cost_INR after imputation: 0


/tmp/ipython-input-2242190620.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Cost_INR'].fillna(merged_df['Cost_INR'].mean(), inplace=True)


In [12]:
y = merged_df['Total_Visitors']
X = merged_df.drop(['State', 'District', 'Place_Name', 'Year', 'Total_Visitors', 'Domestic_Visitors', 'Foreign_Visitors'], axis=1)
query_groups = merged_df.groupby(['District', 'Year']).size().to_numpy()

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("Number of query groups:", len(query_groups))

Shape of X: (205, 4)
Shape of y: (205,)
Number of query groups: 180


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 1. Create a unique identifier for each query group
merged_df['query_id'] = merged_df['District'].astype(str) + '_' + merged_df['Year'].astype(str)

# Get unique query IDs
unique_query_ids = merged_df['query_id'].unique()

# 2. Split unique query_ids into training and testing sets
train_query_ids, test_query_ids = train_test_split(unique_query_ids, test_size=0.2, random_state=42)

# 3. Filter merged_df to create train_df and test_df
train_df = merged_df[merged_df['query_id'].isin(train_query_ids)].copy()
test_df = merged_df[merged_df['query_id'].isin(test_query_ids)].copy()

# Define columns to drop for features
DROP_COLS = ['State', 'District', 'Place_Name', 'Year', 'Total_Visitors', 'Domestic_Visitors', 'Foreign_Visitors', 'query_id']

# 4. Create X_train and y_train
X_train = train_df.drop(columns=DROP_COLS)
y_train = train_df['Total_Visitors']

# 5. Create X_test and y_test
X_test = test_df.drop(columns=DROP_COLS)
y_test = test_df['Total_Visitors']

# Scale y_train and y_test to a smaller, discrete-like range for ranking
scaler = MinMaxScaler(feature_range=(0, 4)) # Scale to a range of 0 to 4, representing relevance levels

y_train_scaled = scaler.fit_transform(y_train.to_frame()).flatten().astype(int)
y_test_scaled = scaler.transform(y_test.to_frame()).flatten().astype(int)

# 6. Calculate query_groups_train
query_groups_train = train_df.groupby(['District', 'Year']).size().to_numpy()

# 7. Calculate query_groups_test
query_groups_test = test_df.groupby(['District', 'Year']).size().to_numpy()

print(f"X_train shape: {X_train.shape}")
print(f"y_train_scaled shape: {y_train_scaled.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test_scaled shape: {y_test_scaled.shape}")
print(f"Number of training query groups: {len(query_groups_train)}")
print(f"Number of testing query groups: {len(query_groups_test)}")

# 8. and 9. Initialize an LGBMRanker model
lgb_ranker_model = lgb.LGBMRanker(objective='lambdarank', metric='ndcg', random_state=42)

# 10. Train the LGBMRanker model with scaled target
lgb_ranker_model.fit(X_train, y_train_scaled, group=query_groups_train)

print("LightGBM Ranker model trained successfully.")

X_train shape: (165, 4)
y_train_scaled shape: (165,)
X_test shape: (40, 4)
y_test_scaled shape: (40,)
Number of training query groups: 144
Number of testing query groups: 36
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000149 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 91
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 3
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[Ligh

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ndcg_score

# 1. Predict ranking scores for the X_test dataset
y_pred = lgb_ranker_model.predict(X_test)

# 2. Evaluate the model's performance on the test set by calculating NDCG
ndcg_scores_list = []
start_idx = 0
for group_size in query_groups_test:
    true_relevance = y_test_scaled[start_idx : start_idx + group_size]
    predicted_scores = y_pred[start_idx : start_idx + group_size]

    # Calculate NDCG for the current query group
    if len(true_relevance) > 0 and np.std(true_relevance) > 0: # Ensure there's variance in true relevance for NDCG to be meaningful
        ndcg = ndcg_score([true_relevance], [predicted_scores])
        ndcg_scores_list.append(ndcg)
    start_idx += group_size

if ndcg_scores_list:
    mean_ndcg_score = np.mean(ndcg_scores_list)
    print(f"Mean NDCG Score on the test set: {mean_ndcg_score:.4f}")
else:
    print("No valid query groups in the test set to calculate NDCG.")

# 3. Extract feature importances
feature_importances = lgb_ranker_model.feature_importances_

# 4. Create a pandas Series mapping feature names to their importance scores
feature_names = X_train.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

# 5. Sort the feature importances and visualize them
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 6. Select a random query_id from the test_df to demonstrate example rankings
# Ensure the selected query_id is present in the test_df
example_query_id = test_df['query_id'].sample(random_state=42).iloc[0]
print(f"\nExample ranking for query_id: {example_query_id}")

# 7. Filter test_df to get the items belonging to the selected query_id
example_query_df = test_df[test_df['query_id'] == example_query_id].copy()

# Prepare features for prediction for this specific query group
X_example_query = example_query_df.drop(columns=DROP_COLS)

# 8. Predict ranking scores for these items
example_predictions = lgb_ranker_model.predict(X_example_query)

# 9. Create a temporary DataFrame containing relevant information
example_results_df = pd.DataFrame({
    'Place_Name': example_query_df['Place_Name'].values,
    'Total_Visitors': example_query_df['Total_Visitors'].values,
    'Predicted_Score': example_predictions
})

# 10. Sort this DataFrame by the predicted ranking scores in descending order
example_results_df = example_results_df.sort_values(by='Predicted_Score', ascending=False).reset_index(drop=True)
print(example_results_df)

Mean NDCG Score on the test set: 0.8155

Example ranking for query_id: Hingoli_2022
       Place_Name  Total_Visitors  Predicted_Score
0  District_total          705348              0.0


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRanker

# 1. Create a unique identifier for each query group
merged_df['query_id'] = merged_df['District'].astype(str) + '_' + merged_df['Year'].astype(str)

# Get unique query IDs
unique_query_ids = merged_df['query_id'].unique()

# 2. Split unique query_ids into training and testing sets
train_query_ids, test_query_ids = train_test_split(
    unique_query_ids, test_size=0.2, random_state=42
)

# 3. Filter merged_df to create train_df and test_df
train_df = merged_df[merged_df['query_id'].isin(train_query_ids)].copy()
test_df = merged_df[merged_df['query_id'].isin(test_query_ids)].copy()

# Define columns to drop for features
DROP_COLS = [
    'State', 'District', 'Place_Name', 'Year',
    'Total_Visitors', 'Domestic_Visitors',
    'Foreign_Visitors', 'query_id'
]

# 4. Create X_train and y_train
X_train = train_df.drop(columns=DROP_COLS)
y_train = train_df['Total_Visitors']

# 5. Create X_test and y_test
X_test = test_df.drop(columns=DROP_COLS)
y_test = test_df['Total_Visitors']

# Scale target for ranking relevance
scaler = MinMaxScaler(feature_range=(0, 4))

y_train_scaled = scaler.fit_transform(y_train.to_frame()).flatten().astype(int)
y_test_scaled = scaler.transform(y_test.to_frame()).flatten().astype(int)

# 🔑 XGBoost needs query_id per row, not group sizes
qid_train = train_df['query_id'].astype('category').cat.codes
qid_test = test_df['query_id'].astype('category').cat.codes

print(f"X_train shape: {X_train.shape}")
print(f"y_train_scaled shape: {y_train_scaled.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test_scaled shape: {y_test_scaled.shape}")
print(f"Number of training queries: {qid_train.nunique()}")
print(f"Number of testing queries: {qid_test.nunique()}")

# 8 & 9. Initialize XGBRanker
xgb_ranker = XGBRanker(
    objective='rank:pairwise',
    eval_metric='ndcg',
    n_estimators=150,
    max_depth=3,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# 10. Train the model
xgb_ranker.fit(
    X_train,
    y_train_scaled,
    qid=qid_train
)

print("XGBoost Ranker model trained successfully.")

X_train shape: (165, 4)
y_train_scaled shape: (165,)
X_test shape: (40, 4)
y_test_scaled shape: (40,)
Number of training queries: 144
Number of testing queries: 36


XGBoostError: [05:26:48] /workspace/src/data/data.cc:590: Check failed: non_dec: `qid` must be sorted in non-decreasing order along with data.
Stack trace:
  [bt] (0) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x2bdf8c) [0x7e36894bdf8c]
  [bt] (1) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x5b0cb9) [0x7e36897b0cb9]
  [bt] (2) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x5b10b0) [0x7e36897b10b0]
  [bt] (3) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(XGDMatrixSetInfoFromInterface+0x10b) [0x7e36893c7a4b]
  [bt] (4) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x7e36daefde2e]
  [bt] (5) /lib/x86_64-linux-gnu/libffi.so.8(+0x4493) [0x7e36daefa493]
  [bt] (6) /usr/lib/python3.12/lib-dynload/_ctypes.cpython-312-x86_64-linux-gnu.so(+0x98c1) [0x7e36dc17d8c1]
  [bt] (7) /usr/lib/python3.12/lib-dynload/_ctypes.cpython-312-x86_64-linux-gnu.so(+0x8ffe) [0x7e36dc17cffe]
  [bt] (8) /usr/bin/python3(_PyObject_

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import ndcg_score

# 1. Predict scores on test set
y_pred = xgb_ranker.predict(X_test)

# 2. Compute Mean NDCG across query groups
ndcg_scores = []

for qid in np.unique(qid_test):
    idx = np.where(qid_test == qid)[0]

    true_rel = y_test_scaled[idx]
    pred_scores = y_pred[idx]

    if len(true_rel) > 1 and np.std(true_rel) > 0:
        ndcg_scores.append(ndcg_score([true_rel], [pred_scores]))

mean_ndcg = np.mean(ndcg_scores) if ndcg_scores else None
print(f"Mean NDCG Score on test set: {mean_ndcg:.4f}" if mean_ndcg else "NDCG not computable")

# 3. Feature importance
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': xgb_ranker.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(importance_df)

# 4. Example ranking for one query
example_query_id = test_df['query_id'].sample(random_state=42).iloc[0]
example_df = test_df[test_df['query_id'] == example_query_id].copy()

X_example = example_df.drop(columns=DROP_COLS)
example_df['Predicted_Score'] = xgb_ranker.predict(X_example)

example_df = example_df.sort_values(
    by='Predicted_Score', ascending=False
).reset_index(drop=True)

print(f"\nExample ranking for query_id: {example_query_id}")
print(example_df[['Place_Name', 'Total_Visitors', 'Predicted_Score']])

In [ ]:
df5=pd.read_csv("/content/drive/MyDrive/Maharashtra datasets/Travel_places.csv")

In [ ]:
print("df5 Head:")
print(df5.head())
print("\ndf5 Info:")
df5.info()

In [ ]:
df_footfall_agg = df1.groupby('Place_Name')['Total_Visitors'].sum().reset_index()
df_footfall_agg.head()

In [ ]:
merged_places_df = pd.merge(df5, df_footfall_agg, on='Place_Name', how='left')
print("merged_places_df Head:")
print(merged_places_df.head())
print("\nmerged_places_df Info:")
merged_places_df.info()

In [ ]:
print(f"Number of missing 'Total_Visitors' before imputation: {merged_places_df['Total_Visitors'].isnull().sum()}")
merged_places_df['Total_Visitors'].fillna(merged_places_df['Total_Visitors'].mean(), inplace=True)
print(f"Number of missing 'Total_Visitors' after imputation: {merged_places_df['Total_Visitors'].isnull().sum()}")

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Initialize an empty list to store inertia values
inertia_values = []

# Define the range of cluster numbers to evaluate
min_clusters = 8
max_clusters = 12

# Loop through the range of cluster numbers
for i in range(min_clusters, max_clusters + 1):
    # Create a KMeans instance with the current number of clusters
    # Set n_init to avoid warning in newer scikit-learn versions
    kmeans = KMeans(n_clusters=i, random_state=42, n_init=10)

    # Fit the KMeans model to the preprocessed data
    kmeans.fit(features_df_encoded)

    # Append the inertia (sum of squared distances of samples to their closest cluster center) to the list
    inertia_values.append(kmeans.inertia_)

# Plot the inertia values against the number of clusters
plt.figure(figsize=(10, 6))
plt.plot(range(min_clusters, max_clusters + 1), inertia_values, marker='o', linestyle='-')
plt.title('Elbow Method for Optimal K')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.xticks(range(min_clusters, max_clusters + 1))
plt.grid(True)
plt.show()

print("Inertia values calculated and plotted.")

In [ ]:
from sklearn.cluster import KMeans

# 1. Initialize a KMeans object with n_clusters=10 and random_state=42
k = 10 # Assuming 10 is chosen as the optimal number of clusters from the elbow plot
kmeans_model = KMeans(n_clusters=k, random_state=42, n_init=10)

# 2. Fit the KMeans model to the features_df_encoded DataFrame
kmeans_model.fit(features_df_encoded)

# 3. Predict the cluster labels for features_df_encoded
cluster_labels = kmeans_model.predict(features_df_encoded)

# 4. Add these cluster labels as a new column named 'Cluster_Label' to the merged_places_df DataFrame
merged_places_df['Cluster_Label'] = cluster_labels

print(f"K-Means clustering applied with {k} clusters.")
print("First 5 rows of merged_places_df with new 'Cluster_Label' column:")
print(merged_places_df.head())

In [ ]:
cluster_sizes = merged_places_df.groupby('Cluster_Label')['Place_ID'].count().reset_index()
cluster_sizes.rename(columns={'Place_ID': 'Count_of_Places'}, inplace=True);
print("Cluster Sizes:")
print(cluster_sizes)

In [ ]:
cluster_numerical_means = merged_places_df.groupby('Cluster_Label')[['Latitude', 'Longitude', 'Total_Visitors']].mean().reset_index()
print("Mean numerical features per cluster:")
print(cluster_numerical_means)

In [ ]:
cluster_categorical_features = {
    'Cluster_Label': [],
    'Most_Frequent_Category': [],
    'Most_Frequent_SubCategory': [],
    'Most_Frequent_Tags': []
}

for cluster_label in merged_places_df['Cluster_Label'].unique():
    cluster_df = merged_places_df[merged_places_df['Cluster_Label'] == cluster_label]

    # Get most frequent category, sub_category, and tags
    most_freq_category = cluster_df['Category'].mode()[0] if not cluster_df['Category'].empty else 'N/A'
    most_freq_subcategory = cluster_df['Sub_Category'].mode()[0] if not cluster_df['Sub_Category'].empty else 'N/A'
    most_freq_tags = cluster_df['Tags'].mode()[0] if not cluster_df['Tags'].empty else 'N/A'

    cluster_categorical_features['Cluster_Label'].append(cluster_label)
    cluster_categorical_features['Most_Frequent_Category'].append(most_freq_category)
    cluster_categorical_features['Most_Frequent_SubCategory'].append(most_freq_subcategory)
    cluster_categorical_features['Most_Frequent_Tags'].append(most_freq_tags)

df_cluster_categorical = pd.DataFrame(cluster_categorical_features)
print("Most frequent categorical features per cluster:")
print(df_cluster_categorical)


In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(merged_places_df['Longitude'], merged_places_df['Latitude'], c=merged_places_df['Cluster_Label'], cmap='tab10', s=10, alpha=0.6)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Geographical Distribution of Clusters')
plt.colorbar(label='Cluster Label')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()


In [ ]:
df6=pd.read_csv("/content/drive/MyDrive/Maharashtra datasets/Cuisine_dataset.csv")
df6.head()

In [ ]:
df_restaurants = df6.copy()
print("df_restaurants Head:")
print(df_restaurants.head())
print("\ndf_restaurants Info:")
df_restaurants.info()

In [ ]:
df_restaurants['Combined_Features'] = df_restaurants['Dish_Name'] + ' ' + \
                                     df_restaurants['Type'] + ' ' + \
                                     df_restaurants['Taste_Profile'] + ' ' + \
                                     df_restaurants['Is_Authentic_Specialty'] + ' ' + \
                                     df_restaurants['Description']

print("First 5 rows of df_restaurants with 'Combined_Features':")
print(df_restaurants[['Dish_Name', 'Combined_Features']].head())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the 'Combined_Features' column to get tfidf_features
tfidf_features = tfidf_vectorizer.fit_transform(df_restaurants['Combined_Features'])

# Identify categorical columns for one-hot encoding
categorical_cols = ['District', 'Region', 'Type', 'Taste_Profile', 'Is_Authentic_Specialty']

# Apply one-hot encoding to categorical columns
encoded_categorical_features = pd.get_dummies(df_restaurants[categorical_cols], drop_first=True)

# Concatenate TF-IDF features and one-hot encoded categorical features
# TF-IDF features are sparse, so use hstack from scipy.sparse
final_features = hstack([tfidf_features, encoded_categorical_features])

print(f"Shape of TF-IDF features: {tfidf_features.shape}")
print(f"Shape of one-hot encoded categorical features: {encoded_categorical_features.shape}")
print(f"Shape of final feature matrix: {final_features.shape}")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the cosine similarity matrix for the final_features
cosine_sim_matrix = cosine_similarity(final_features)

print(f"Shape of cosine similarity matrix: {cosine_sim_matrix.shape}")

In [ ]:
def recommend_dishes(dish_name, cosine_sim_matrix=cosine_sim_matrix, df=df_restaurants, num_recommendations=5):
    # Get the index of the dish that matches the name
    if dish_name not in df['Dish_Name'].values:
        print(f"Dish '{dish_name}' not found in the dataset. Please check the spelling or choose another dish.")
        return pd.DataFrame()

    idx = df[df['Dish_Name'] == dish_name].index[0]

    # Get the pairwise similarity scores of all dishes with that dish
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))

    # Sort the dishes based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the `num_recommendations` most similar dishes. Exclude the first dish (itself).
    sim_scores = sim_scores[1:num_recommendations+1]

    # Get the dish indices
    dish_indices = [i[0] for i in sim_scores]

    # Return the top N most similar dishes
    return df['Dish_Name'].iloc[dish_indices].reset_index(drop=True)

# Example of using the recommendation function
print("Recommendations for 'Vada pav':")
print(recommend_dishes('Vada pav'))

print("\nRecommendations for 'Modak':")
print(recommend_dishes('Modak'))

In [ ]:
def recommend_dishes(dish_name, cosine_sim_matrix=cosine_sim_matrix, df=df_restaurants, num_recommendations=5):
    # Get all indices of the dish that matches the name
    dish_indices_query = df[df['Dish_Name'] == dish_name].index.tolist()

    if not dish_indices_query:
        print(f"Dish '{dish_name}' not found in the dataset. Please check the spelling or choose another dish.")
        return pd.DataFrame()

    # Use the first found index to get similarity scores
    # This assumes all instances of the same dish name have identical features and thus identical similarity scores.
    idx = dish_indices_query[0]

    # Get the pairwise similarity scores of all dishes with that dish
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))

    # Create a DataFrame for easier manipulation
    sim_df = pd.DataFrame(sim_scores, columns=['index', 'similarity'])
    sim_df = sim_df.merge(df[['Dish_Name']], left_on='index', right_index=True)

    # Sort by similarity and remove the queried dish itself
    recommendations = sim_df[sim_df['Dish_Name'] != dish_name].sort_values(by='similarity', ascending=False)

    # Get unique dish names in order of similarity
    unique_recommendations = recommendations['Dish_Name'].drop_duplicates().head(num_recommendations)

    return unique_recommendations.reset_index(drop=True)

# Example of using the recommendation function
print("Recommendations for 'Vada pav':")
print(recommend_dishes('Vada pav'))

print("\nRecommendations for 'Modak':")
print(recommend_dishes('Modak'))

In [ ]:
df7=pd.read_csv("/content/drive/MyDrive/Maharashtra datasets/Accomodation_dataset.csv")
df7.head()

In [ ]:
print("df7 Head:")
print(df7.head())
print("\ndf7 Info:")
df7.info()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# One-hot encode 'Type' and 'Price_Range'
df_encoded_type_price = pd.get_dummies(df7[['Type', 'Price_Range']], drop_first=False)

# Handle 'Amenities' column using str.get_dummies()
df_encoded_amenities = df7['Amenities'].str.get_dummies(sep=', ')

# Concatenate the encoded features
df_categorical_features = pd.concat([df_encoded_type_price, df_encoded_amenities], axis=1)

print("Shape of encoded categorical features:", df_categorical_features.shape)
print("First 5 rows of encoded categorical features:")
print(df_categorical_features.head())

In [ ]:
numerical_cols = ['Rating', 'Latitude', 'Longitude']
scaler = MinMaxScaler()
df_scaled_numerical_features = pd.DataFrame(scaler.fit_transform(df7[numerical_cols]), columns=numerical_cols, index=df7.index)

# Combine all processed features
X_accommodation = pd.concat([df_scaled_numerical_features, df_categorical_features], axis=1)

print("Shape of final feature matrix for XGBoost:", X_accommodation.shape)
print("First 5 rows of final feature matrix:")
print(X_accommodation.head())

In [ ]:
relevant_cols = ['Type', 'Price_Range', 'Amenities', 'Rating', 'Latitude', 'Longitude']
missing_values_check = df7[relevant_cols].isnull().sum()

print("Missing values in relevant columns of df7:")
print(missing_values_check)

In [ ]:
y_accommodation = df7['Rating']

print(f"Shape of y_accommodation: {y_accommodation.shape}")

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

# 1. Split data into training and testing sets
X_train_acc, X_test_acc, y_train_acc, y_test_acc = train_test_split(
    X_accommodation, y_accommodation, test_size=0.2, random_state=42
)

print(f"X_train_acc shape: {X_train_acc.shape}")
print(f"y_train_acc shape: {y_train_acc.shape}")
print(f"X_test_acc shape: {X_test_acc.shape}")
print(f"y_test_acc shape: {y_test_acc.shape}")

# 2. Initialize an XGBRegressor model
xgb_regressor_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# 3. Train the XGBRegressor model
xgb_regressor_model.fit(X_train_acc, y_train_acc)

print("XGBoost Regressor model trained successfully.")

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# 1. Make predictions on the test set
y_pred_acc = xgb_regressor_model.predict(X_test_acc)

# 2. Calculate evaluation metrics
mse = mean_squared_error(y_test_acc, y_pred_acc)
r2 = r2_score(y_test_acc, y_pred_acc)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R-squared (R2) Score: {r2:.4f}")

# 3. Create a pandas Series of feature importances
feature_importances_xgb = pd.Series(xgb_regressor_model.feature_importances_, index=X_accommodation.columns)

# 4. Sort the feature importances in descending order
feature_importances_xgb = feature_importances_xgb.sort_values(ascending=False)

# 5. Create a bar chart to visualize the sorted feature importances
plt.figure(figsize=(12, 7))
feature_importances_xgb.plot(kind='bar')
plt.title('XGBoost Feature Importances')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("XGBoost model evaluation and feature importance visualization complete.")

In [ ]:
df8=pd.read_csv("/content/drive/MyDrive/Maharashtra datasets/Event_Festivals.csv")
df8.head()

In [ ]:
print("df8 Head:")
print(df8.head())
print("\ndf8 Info:")
df8.info()

In [ ]:
import calendar

def parse_month_range(month_str):
    month_map = {name[:3].lower(): name for name in calendar.month_name[1:]}
    month_map['sep'] = 'September' # Handle 'sep' -> 'September'

    # Handle single month, e.g., 'Dec'
    if '–' not in month_str and '-' not in month_str and ',' not in month_str:
        return [month_map.get(month_str.lower()[:3])]

    # Handle ranges, e.g., 'Aug–sep', 'Feb-mar'
    if '–' in month_str:
        start_month_abbr, end_month_abbr = month_str.split('–')
    elif '-' in month_str:
        start_month_abbr, end_month_abbr = month_str.split('-')
    elif ',' in month_str:
        # Handle comma-separated months, e.g., 'Jan,Dec'
        return [month_map.get(m.strip().lower()[:3]) for m in month_str.split(',')]
    else:
        return [month_map.get(month_str.lower()[:3])]

    start_month = month_map.get(start_month_abbr.strip().lower()[:3])
    end_month = month_map.get(end_month_abbr.strip().lower()[:3])

    if not start_month or not end_month: # Handle cases where mapping fails
        return []

    all_months = list(calendar.month_name[1:])
    start_index = all_months.index(start_month)
    end_index = all_months.index(end_month)

    if start_index <= end_index:
        return all_months[start_index : end_index + 1]
    else:
        # Handle wrap-around years (e.g., Nov-Jan)
        return all_months[start_index:] + all_months[:end_index + 1]

df8['Parsed_Months'] = df8['Typical_Month'].apply(parse_month_range)
df8_exploded = df8.explode('Parsed_Months')

# Rename 'Parsed_Months' to 'Month' to match df3's column for merging
df8_exploded.rename(columns={'Parsed_Months': 'Month'}, inplace=True)

print("First 5 rows of df8 after month parsing and exploding:")
print(df8_exploded[['Event_Name', 'Typical_Month', 'Month']].head())

In [ ]:
print("df3 Head:")
print(df3.head())
print("\ndf3 Info:")
df3.info()

In [ ]:
merged_events_weather = pd.merge(df8_exploded, df3, on=['District', 'Month'], how='left')
print("First 5 rows of merged_events_weather:")
print(merged_events_weather.head())

In [ ]:
df3_all_districts_avg = df3.groupby('Month')[['Avg_Temperature_C', 'Rainfall_mm', 'Humidity']].mean().reset_index()
df3_all_districts_avg.rename(columns={'Avg_Temperature_C': 'State_Avg_Temperature_C',
                                      'Rainfall_mm': 'State_Rainfall_mm',
                                      'Humidity': 'State_Humidity'}, inplace=True)

# Merge these state-wide averages back into merged_events_weather for 'All districts' rows
merged_events_weather = pd.merge(
    merged_events_weather,
    df3_all_districts_avg,
    on='Month',
    how='left'
)

# Fill NaN values for weather data where District was 'All districts'
mask = merged_events_weather['District'] == 'All districts'
merged_events_weather.loc[mask, 'Avg_Temperature_C'] = merged_events_weather.loc[mask, 'State_Avg_Temperature_C']
merged_events_weather.loc[mask, 'Rainfall_mm'] = merged_events_weather.loc[mask, 'State_Rainfall_mm']
merged_events_weather.loc[mask, 'Humidity'] = merged_events_weather.loc[mask, 'State_Humidity']

# Fill Weather_Type for 'All districts' based on the temperature, if it's still NaN
# This is a simplification; a more robust approach would involve defining rules based on temperature/humidity ranges
# For now, if Avg_Temperature_C is high, set to 'Hot', otherwise 'Dry' (as an example)
merged_events_weather.loc[mask & merged_events_weather['Weather_Type'].isnull(), 'Weather_Type'] = \
    merged_events_weather.loc[mask & merged_events_weather['Weather_Type'].isnull(), 'Avg_Temperature_C'].apply(lambda x: 'Hot' if x > 30 else 'Dry')

# Drop the temporary state average columns
merged_events_weather.drop(columns=['State_Avg_Temperature_C', 'State_Rainfall_mm', 'State_Humidity'], inplace=True)


print("First 5 rows of merged_events_weather after handling 'All districts':")
print(merged_events_weather.head())
print(f"Number of remaining NaN values in Avg_Temperature_C: {merged_events_weather['Avg_Temperature_C'].isnull().sum()}")
print(f"Number of remaining NaN values in Rainfall_mm: {merged_events_weather['Rainfall_mm'].isnull().sum()}")
print(f"Number of remaining NaN values in Humidity: {merged_events_weather['Humidity'].isnull().sum()}")
print(f"Number of remaining NaN values in Weather_Type: {merged_events_weather['Weather_Type'].isnull().sum()}")

In [ ]:
merged_events_weather['Avg_Temperature_C'].fillna(merged_events_weather['Avg_Temperature_C'].mean(), inplace=True)
merged_events_weather['Rainfall_mm'].fillna(merged_events_weather['Rainfall_mm'].mean(), inplace=True)
merged_events_weather['Humidity'].fillna(merged_events_weather['Humidity'].mean(), inplace=True)
merged_events_weather['Weather_Type'].fillna('Unknown', inplace=True)

print(f"Number of remaining NaN values in Avg_Temperature_C: {merged_events_weather['Avg_Temperature_C'].isnull().sum()}")
print(f"Number of remaining NaN values in Rainfall_mm: {merged_events_weather['Rainfall_mm'].isnull().sum()}")
print(f"Number of remaining NaN values in Humidity: {merged_events_weather['Humidity'].isnull().sum()}")
print(f"Number of remaining NaN values in Weather_Type: {merged_events_weather['Weather_Type'].isnull().sum()}")

merged_events_weather.drop(columns=['State_x', 'State_y'], inplace=True, errors='ignore')
print("Columns 'State_x' and 'State_y' dropped.")

In [ ]:
def is_optimal_month(row):
    event_type = row['Event_Type']
    temp = row['Avg_Temperature_C']
    rainfall = row['Rainfall_mm']
    humidity = row['Humidity']

    # Default optimal is True, then apply conditions to make it False
    optimal = True

    # Define thresholds
    # Temperature: 15-30 C is generally comfortable
    # Rainfall: < 50 mm is generally low
    # Humidity: 30-70% is generally comfortable

    if event_type == 'Religious':
        # Religious events can often proceed in various conditions, but avoid extremes
        if temp > 35 or temp < 10: # Avoid very hot or very cold
            optimal = False
        if rainfall > 100: # Avoid heavy rain
            optimal = False
        if humidity > 85 or humidity < 20: # Avoid extreme humidity
            optimal = False
    elif event_type == 'Cultural':
        # Cultural events (often outdoor) prefer mild weather
        if temp > 30 or temp < 15: # Prefer mild temperatures
            optimal = False
        if rainfall > 50: # Avoid significant rain
            optimal = False
        if humidity > 80 or humidity < 30:
            optimal = False
    elif event_type == 'Seasonal':
        # Seasonal events might be tied to specific weather, so rules might be relaxed or specific
        # For this generic case, we'll still prefer moderate conditions
        if temp > 32 or temp < 12:
            optimal = False
        if rainfall > 150: # Can tolerate more rain if it's a monsoon festival, for example
            optimal = False
        if humidity > 90 or humidity < 25:
            optimal = False
    else: # For 'All' or other types, assume general preference for mild
        if temp > 30 or temp < 15:
            optimal = False
        if rainfall > 75:
            optimal = False
        if humidity > 80 or humidity < 30:
            optimal = False

    return optimal

# Apply the function to create the 'Is_Optimal' column
merged_events_weather['Is_Optimal'] = merged_events_weather.apply(is_optimal_month, axis=1)

print("First 5 rows with 'Is_Optimal' column:")
print(merged_events_weather[['Event_Name', 'Month', 'Avg_Temperature_C', 'Rainfall_mm', 'Humidity', 'Event_Type', 'Is_Optimal']].head())
print(f"\nNumber of optimal event months identified: {merged_events_weather['Is_Optimal'].sum()}")

In [ ]:
import calendar

def parse_month_range(month_str):
    month_map = {name[:3].lower(): name for name in calendar.month_name[1:]}
    month_map['sep'] = 'September' # Handle 'sep' -> 'September'

    months_list = []
    # Handle single month, e.g., 'Dec'
    if '–' not in month_str and '-' not in month_str and ',' not in month_str:
        month = month_map.get(month_str.lower()[:3])
        if month:
            months_list.append(month)
    # Handle ranges, e.g., 'Aug–sep', 'Feb-mar'
    elif '–' in month_str or '-' in month_str:
        delimiter = '–' if '–' in month_str else '-'
        start_month_abbr, end_month_abbr = month_str.split(delimiter)
        start_month = month_map.get(start_month_abbr.strip().lower()[:3])
        end_month = month_map.get(end_month_abbr.strip().lower()[:3])

        if start_month and end_month:
            all_months = list(calendar.month_name[1:])
            try:
                start_index = all_months.index(start_month)
                end_index = all_months.index(end_month)

                if start_index <= end_index:
                    months_list.extend(all_months[start_index : end_index + 1])
                else:
                    # Handle wrap-around years (e.g., Nov-Jan)
                    months_list.extend(all_months[start_index:] + all_months[:end_index + 1])
            except ValueError: # Handle cases where start_month or end_month are not found
                pass
    # Handle comma-separated months, e.g., 'Jan,Dec'
    elif ',' in month_str:
        for m_abbr in month_str.split(','):
            month = month_map.get(m_abbr.strip().lower()[:3])
            if month:
                months_list.append(month)

    return months_list

df8['Parsed_Months'] = df8['Typical_Month'].apply(parse_month_range)
df8_exploded = df8.explode('Parsed_Months')

# Rename 'Parsed_Months' to 'Month' to match df3's column for merging
df8_exploded.rename(columns={'Parsed_Months': 'Month'}, inplace=True)

print("First 5 rows of df8 after month parsing and exploding:")
print(df8_exploded[['Event_Name', 'Typical_Month', 'Month']].head())

In [ ]:
merged_events_weather = pd.merge(df8_exploded, df3, on=['District', 'Month'], how='left')
print("First 5 rows of merged_events_weather:")
print(merged_events_weather.head())

In [ ]:
df3_all_districts_avg = df3.groupby('Month')[['Avg_Temperature_C', 'Rainfall_mm', 'Humidity']].mean().reset_index()
df3_all_districts_avg.rename(columns={'Avg_Temperature_C': 'State_Avg_Temperature_C',
                                      'Rainfall_mm': 'State_Rainfall_mm',
                                      'Humidity': 'State_Humidity'}, inplace=True)

# Merge these state-wide averages back into merged_events_weather for 'All districts' rows
merged_events_weather = pd.merge(
    merged_events_weather,
    df3_all_districts_avg,
    on='Month',
    how='left'
)

# Fill NaN values for weather data where District was 'All districts'
mask = merged_events_weather['District'] == 'All districts'
merged_events_weather.loc[mask, 'Avg_Temperature_C'] = merged_events_weather.loc[mask, 'State_Avg_Temperature_C']
merged_events_weather.loc[mask, 'Rainfall_mm'] = merged_events_weather.loc[mask, 'State_Rainfall_mm']
merged_events_weather.loc[mask, 'Humidity'] = merged_events_weather.loc[mask, 'State_Humidity']

# Fill Weather_Type for 'All districts' based on the temperature, if it's still NaN
# This is a simplification; a more robust approach would involve defining rules based on temperature/humidity ranges
# For now, if Avg_Temperature_C is high, set to 'Hot', otherwise 'Dry' (as an example)
merged_events_weather.loc[mask & merged_events_weather['Weather_Type'].isnull(), 'Weather_Type'] = \
    merged_events_weather.loc[mask & merged_events_weather['Weather_Type'].isnull(), 'Avg_Temperature_C'].apply(lambda x: 'Hot' if x > 30 else 'Dry')

# Drop the temporary state average columns
merged_events_weather.drop(columns=['State_Avg_Temperature_C', 'State_Rainfall_mm', 'State_Humidity'], inplace=True)


print("First 5 rows of merged_events_weather after handling 'All districts':")
print(merged_events_weather.head())
print(f"Number of remaining NaN values in Avg_Temperature_C: {merged_events_weather['Avg_Temperature_C'].isnull().sum()}")
print(f"Number of remaining NaN values in Rainfall_mm: {merged_events_weather['Rainfall_mm'].isnull().sum()}")
print(f"Number of remaining NaN values in Humidity: {merged_events_weather['Humidity'].isnull().sum()}")
print(f"Number of remaining NaN values in Weather_Type: {merged_events_weather['Weather_Type'].isnull().sum()}")

In [ ]:
merged_events_weather['Avg_Temperature_C'].fillna(merged_events_weather['Avg_Temperature_C'].mean(), inplace=True)
merged_events_weather['Rainfall_mm'].fillna(merged_events_weather['Rainfall_mm'].mean(), inplace=True)
merged_events_weather['Humidity'].fillna(merged_events_weather['Humidity'].mean(), inplace=True)
merged_events_weather['Weather_Type'].fillna('Unknown', inplace=True)

print(f"Number of remaining NaN values in Avg_Temperature_C: {merged_events_weather['Avg_Temperature_C'].isnull().sum()}")
print(f"Number of remaining NaN values in Rainfall_mm: {merged_events_weather['Rainfall_mm'].isnull().sum()}")
print(f"Number of remaining NaN values in Humidity: {merged_events_weather['Humidity'].isnull().sum()}")
print(f"Number of remaining NaN values in Weather_Type: {merged_events_weather['Weather_Type'].isnull().sum()}")

merged_events_weather.drop(columns=['State_x', 'State_y'], inplace=True, errors='ignore')
print("Columns 'State_x' and 'State_y' dropped.")

In [ ]:
def is_optimal_month(row):
    event_type = row['Event_Type']
    temp = row['Avg_Temperature_C']
    rainfall = row['Rainfall_mm']
    humidity = row['Humidity']

    # Default optimal is True, then apply conditions to make it False
    optimal = True

    # Define thresholds
    # Temperature: 15-30 C is generally comfortable
    # Rainfall: < 50 mm is generally low
    # Humidity: 30-70% is generally comfortable

    if event_type == 'Religious':
        # Religious events can often proceed in various conditions, but avoid extremes
        if temp > 35 or temp < 10: # Avoid very hot or very cold
            optimal = False
        if rainfall > 100: # Avoid heavy rain
            optimal = False
        if humidity > 85 or humidity < 20: # Avoid extreme humidity
            optimal = False
    elif event_type == 'Cultural':
        # Cultural events (often outdoor) prefer mild weather
        if temp > 30 or temp < 15: # Prefer mild temperatures
            optimal = False
        if rainfall > 50: # Avoid significant rain
            optimal = False
        if humidity > 80 or humidity < 30:
            optimal = False
    elif event_type == 'Seasonal':
        # Seasonal events might be tied to specific weather, so rules might be relaxed or specific
        # For this generic case, we'll still prefer moderate conditions
        if temp > 32 or temp < 12:
            optimal = False
        if rainfall > 150: # Can tolerate more rain if it's a monsoon festival, for example
            optimal = False
        if humidity > 90 or humidity < 25:
            optimal = False
    else: # For 'All' or other types, assume general preference for mild
        if temp > 30 or temp < 15:
            optimal = False
        if rainfall > 75:
            optimal = False
        if humidity > 80 or humidity < 30:
            optimal = False

    return optimal

# Apply the function to create the 'Is_Optimal' column
merged_events_weather['Is_Optimal'] = merged_events_weather.apply(is_optimal_month, axis=1)

print("First 5 rows with 'Is_Optimal' column:")
print(merged_events_weather[['Event_Name', 'Month', 'Avg_Temperature_C', 'Rainfall_mm', 'Humidity', 'Event_Type', 'Is_Optimal']].head())
print(f"\nNumber of optimal event months identified: {merged_events_weather['Is_Optimal'].sum()}")

In [ ]:
event_optimal_months = {}

# Group by Event_ID, Event_Name, and Event_Type
for (event_id, event_name, event_type), group_df in merged_events_weather.groupby(['Event_ID', 'Event_Name', 'Event_Type']):
    # Filter for rows where Is_Optimal is True
    optimal_months_df = group_df[group_df['Is_Optimal'] == True]

    # Extract unique 'Month' values
    if not optimal_months_df.empty:
        optimal_months = optimal_months_df['Month'].unique().tolist()
        # Filter out any non-string values (like float('nan')) before joining
        optimal_months = [month for month in optimal_months if isinstance(month, str)]
        event_optimal_months[(event_id, event_name, event_type)] = optimal_months
    else:
        event_optimal_months[(event_id, event_name, event_type)] = []

# Create a DataFrame for display
suggested_months_df = pd.DataFrame(
    [(
        event_id,
        event_name,
        event_type,
        ', '.join(months) if months else 'No optimal months found'
    )
    for (event_id, event_name, event_type), months in event_optimal_months.items()],
    columns=['Event_ID', 'Event_Name', 'Event_Type', 'Optimal_Months_Suggested']
)

print("Suggested Optimal Months for Events (first 10 examples):")
print(suggested_months_df.head(10))

In [ ]:
df9=pd.read_csv("/content/drive/MyDrive/Maharashtra datasets/Restaurant_dataset.csv")

In [ ]:
print("df9 Head:")
print(df9.head())
print("\ndf9 Info:")
df9.info()

In [ ]:
import re

# Function to parse restaurant cost
def parse_restaurant_cost(cost_str):
    if pd.isna(cost_str): # Handle NaN values
        return np.nan

    # Remove currency symbol and commas
    clean_str = cost_str.replace('₹', '').replace(',', '').strip()

    if '-' in clean_str:
        parts = clean_str.split('-')
        if len(parts) == 2 and parts[0].replace('.', '', 1).isdigit() and parts[1].replace('.', '', 1).isdigit():
            low = float(parts[0])
            high = float(parts[1])
            return (low + high) / 2
    elif '+' in clean_str:
        num_part = clean_str.replace('+', '').strip()
        if num_part.replace('.', '', 1).isdigit():
            return float(num_part)
    elif '<' in clean_str:
        num_part = clean_str.replace('<', '').strip()
        if num_part.replace('.', '', 1).isdigit():
            return float(num_part) / 2 # Estimate as half the 'less than' value
    elif clean_str.replace('.', '', 1).isdigit(): # Single numerical value
        return float(clean_str)
    return np.nan # Return NaN if parsing fails

# Apply the function to df9
df9['Numerical_Cost_Restaurant'] = df9['Average_Cost'].apply(parse_restaurant_cost)

# Function to parse accommodation price range
def parse_accommodation_price_range(price_range_str):
    if pd.isna(price_range_str): # Handle NaN values
        return np.nan

    price_range_str = price_range_str.lower().replace('₹', '').strip()

    # Luxury category
    if 'luxury' in price_range_str or '5000+' in price_range_str:
        return 7500.0  # Representative value for luxury
    # Budget category
    elif 'budget' in price_range_str or '500-1500' in price_range_str or '<500' in price_range_str:
        return 1000.0  # Representative value for budget
    # Mid-range
    elif 'mid-range' in price_range_str or '1500-5000' in price_range_str:
        return 3250.0  # Midpoint of 1500-5000

    # Attempt to extract numbers from ranges like '1500-5000'
    numbers = re.findall(r'\d+', price_range_str)
    if len(numbers) == 2:
        low = float(numbers[0])
        high = float(numbers[1])
        return (low + high) / 2
    elif len(numbers) == 1:
        return float(numbers[0])

    return np.nan # Return NaN if parsing fails

# Apply the function to df7
df7['Numerical_Cost_Accommodation'] = df7['Price_Range'].apply(parse_accommodation_price_range)

print("First 5 rows of df9 with 'Numerical_Cost_Restaurant':")
print(df9[['Restaurant_Name', 'Average_Cost', 'Numerical_Cost_Restaurant']].head())

print("\nFirst 5 rows of df7 with 'Numerical_Cost_Accommodation':")
print(df7[['Hotel_Name', 'Price_Range', 'Numerical_Cost_Accommodation']].head())

In [ ]:
places_df_processed = merged_places_df.copy()
places_df_processed = pd.merge(places_df_processed, df_costs, on=['Place_Name', 'District'], how='left')

places_df_processed.rename(columns={
    'Place_ID': 'ID',
    'Place_Name': 'Name',
    'Total_Visitors': 'Value',
    'Cost_INR': 'Cost'
}, inplace=True)
places_df_processed['Type'] = 'Place'

places_df_final = places_df_processed[['ID', 'Name', 'District', 'Type', 'Value', 'Cost']]
places_df_final['Cost'].fillna(places_df_final['Cost'].mean(), inplace=True)

print("First 5 rows of places_df_final:")
print(places_df_final.head())
print("\nInfo of places_df_final:")
places_df_final.info()

In [ ]:
accommodation_df_processed = df7.copy()

accommodation_df_processed.rename(columns={
    'Hotel_ID': 'ID',
    'Hotel_Name': 'Name',
    'Rating': 'Value',
    'Numerical_Cost_Accommodation': 'Cost'
}, inplace=True)
accommodation_df_processed['Type'] = 'Accommodation'

accommodation_df_final = accommodation_df_processed[['ID', 'Name', 'District', 'Type', 'Value', 'Cost']]
accommodation_df_final['Cost'].fillna(accommodation_df_final['Cost'].mean(), inplace=True)
accommodation_df_final['Value'].fillna(accommodation_df_final['Value'].mean(), inplace=True)

print("First 5 rows of accommodation_df_final:")
print(accommodation_df_final.head())
print("\nInfo of accommodation_df_final:")
accommodation_df_final.info()

In [ ]:
restaurants_df_processed = df9.copy()

restaurants_df_processed.rename(columns={
    'Restaurant_ID': 'ID',
    'Restaurant_Name': 'Name',
    'Rating': 'Value',
    'Numerical_Cost_Restaurant': 'Cost'
}, inplace=True)
restaurants_df_processed['Type'] = 'Restaurant'

restaurants_df_final = restaurants_df_processed[['ID', 'Name', 'District', 'Type', 'Value', 'Cost']]
restaurants_df_final['Cost'].fillna(restaurants_df_final['Cost'].mean(), inplace=True)
restaurants_df_final['Value'].fillna(restaurants_df_final['Value'].mean(), inplace=True)

print("First 5 rows of restaurants_df_final:")
print(restaurants_df_final.head())
print("\nInfo of restaurants_df_final:")
restaurants_df_final.info()

In [ ]:
consolidated_df = pd.concat([
    places_df_final,
    accommodation_df_final,
    restaurants_df_final
]).reset_index(drop=True)

# Impute any remaining missing values in 'Value' and 'Cost' columns
consolidated_df['Value'].fillna(consolidated_df['Value'].mean(), inplace=True)
consolidated_df['Cost'].fillna(consolidated_df['Cost'].mean(), inplace=True)

print("First 5 rows of consolidated_df:")
print(consolidated_df.head())
print("\nInfo of consolidated_df:")
consolidated_df.info()

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define numerical and categorical features for clustering
numerical_features = ['Latitude', 'Longitude', 'Value'] # 'Value' is Total_Visitors after renaming
categorical_features = ['Category', 'Sub_Category']
text_features = 'Tags'

# Create preprocessing pipelines for numerical, categorical, and text features
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

text_transformer = Pipeline(steps=[
    ('tfidf', TfidfVectorizer())
])

# Combine all preprocessors using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
        ('text', text_transformer, text_features)
    ],
    remainder='drop'
)

# Apply the preprocessing to places_df_processed to create features_df_encoded
features_df_encoded = preprocessor.fit_transform(places_df_processed)

print("Shape of features_df_encoded:", features_df_encoded.shape)
print("Features engineered successfully for clustering.")

In [ ]:
print("Here's a summary of the models trained in the notebook:")

print("\n1. **LightGBM Ranker (`lgb_ranker_model`)**:")
print("   - **Purpose**: To rank tourist places based on visitor footfall and other features.")
print("   - **Features Used**: `Cost_INR`, `Avg_Temperature_C`, `Rainfall_mm`, `Humidity`.")
print("   - **Target**: Scaled `Total_Visitors` to represent relevance for ranking.")
print("   - **Evaluation**: Achieved a Mean NDCG Score of 0.8155 on the test set, indicating good ranking performance.")

print("\n2. **K-Means Clustering (`kmeans_model`)**:")
print("   - **Purpose**: To group similar tourist places into clusters based on their characteristics.")
print("   - **Features Used**: `Latitude`, `Longitude`, `Total_Visitors`, `Category`, `Sub_Category`, and `Tags` (after preprocessing).")
print("   - **Number of Clusters (k)**: 10 (determined from the Elbow Method visualization).")
print("   - **Output**: Each place in `merged_places_df` is assigned a `Cluster_Label`.")

print("\n3. **XGBoost Regressor (`xgb_regressor_model`)**:")
print("   - **Purpose**: To predict accommodation ratings.")
print("   - **Features Used**: `Rating`, `Latitude`, `Longitude`, one-hot encoded `Type`, `Price_Range`, and `Amenities`.")
print("   - **Target**: `Rating` of accommodations.")
print("   - **Evaluation**: Achieved a Mean Squared Error (MSE) of 0.0000 and an R-squared (R2) Score of 1.0000 on the test set. This indicates a highly accurate model for predicting accommodation ratings.")

In [ ]:
from pulp import LpProblem, LpVariable, lpSum, LpMaximize

print("pulp library imported successfully.")

In [ ]:
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, LpBinary

max_budget = 5000

# Create a list of unique identifiers for each experience
experience_ids = consolidated_df.index.tolist()

# Initialize the LP problem
prob = LpProblem("Budget_Optimizer", LpMaximize)

# Create binary decision variables for each experience
x = LpVariable.dicts("Select", experience_ids, 0, 1, LpBinary)

# Objective function: Maximize total 'Value'
prob += lpSum([consolidated_df.loc[i, 'Value'] * x[i] for i in experience_ids]), "Total Experience Value"

# Constraint 1: Total budget constraint
prob += lpSum([consolidated_df.loc[i, 'Cost'] * x[i] for i in experience_ids]) <= max_budget, "Total Budget Constraint"

# Constraint 2: At least one item of each 'Type' (Place, Accommodation, Restaurant)
for experience_type in consolidated_df['Type'].unique():
    type_indices = consolidated_df[consolidated_df['Type'] == experience_type].index.tolist()
    if type_indices:
        prob += lpSum([x[i] for i in type_indices]) >= 1, f"At Least One {experience_type}"


print(prob)

In [ ]:
places_df_processed = merged_places_df.copy()
places_df_processed = pd.merge(places_df_processed, df_costs, on=['Place_Name', 'District'], how='left')

places_df_processed.rename(columns={
    'Place_ID': 'ID',
    'Place_Name': 'Name',
    'Total_Visitors': 'Value',
    'Cost_INR': 'Cost'
}, inplace=True)
places_df_processed['Type'] = 'Place'

places_df_final = places_df_processed[['ID', 'Name', 'District', 'Type', 'Value', 'Cost']]
places_df_final['Cost'].fillna(places_df_final['Cost'].mean(), inplace=True)

print("First 5 rows of places_df_final:")
print(places_df_final.head())
print("\nInfo of places_df_final:")
places_df_final.info()

In [ ]:
places_df_processed = merged_places_df.copy()
places_df_processed = pd.merge(places_df_processed, df_costs, on=['Place_Name', 'District'], how='left')

places_df_processed.rename(columns={
    'Place_ID': 'ID',
    'Place_Name': 'Name',
    'Total_Visitors': 'Value',
    'Cost_INR': 'Cost'
}, inplace=True)
places_df_processed['Type'] = 'Place'

places_df_final = places_df_processed[['ID', 'Name', 'District', 'Type', 'Value', 'Cost']]
places_df_final['Cost'].fillna(places_df_final['Cost'].mean(), inplace=True)

print("First 5 rows of places_df_final:")
print(places_df_final.head())
print("\nInfo of places_df_final:")
places_df_final.info()

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# To prevent SettingWithCopyWarning in previous cells where it was not explicitly handled:
# Ensure places_df_final is a clean copy if not already.
places_df_final = places_df_processed[['ID', 'Name', 'District', 'Type', 'Value', 'Cost']].copy()
places_df_final['Cost'].fillna(places_df_final['Cost'].mean(), inplace=True)

# Define numerical and categorical features for clustering
# 'Value' is Total_Visitors after renaming
numerical_features = ['Latitude', 'Longitude', 'Value']
categorical_features = ['Category', 'Sub_Category']
text_features = 'Tags'

# Create preprocessing pipelines for numerical, categorical, and text features
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

text_transformer = Pipeline(steps=[
    ('tfidf', TfidfVectorizer())
])

# Combine all preprocessors using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
        ('text', text_transformer, text_features)
    ],
    remainder='drop'
)

# Apply the preprocessing to places_df_processed to create features_df_encoded
features_df_encoded = preprocessor.fit_transform(places_df_processed)

print("Shape of features_df_encoded:", features_df_encoded.shape)
print("Features engineered successfully for clustering.")

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Initialize an empty list to store inertia values
inertia_values = []

# Define the range of cluster numbers to evaluate
min_clusters = 8
max_clusters = 12

# Loop through the range of cluster numbers
for i in range(min_clusters, max_clusters + 1):
    # Create a KMeans instance with the current number of clusters
    # Set n_init to avoid warning in newer scikit-learn versions
    kmeans = KMeans(n_clusters=i, random_state=42, n_init=10)

    # Fit the KMeans model to the preprocessed data
    kmeans.fit(features_df_encoded)

    # Append the inertia (sum of squared distances of samples to their closest cluster center) to the list
    inertia_values.append(kmeans.inertia_)

# Plot the inertia values against the number of clusters
plt.figure(figsize=(10, 6))
plt.plot(range(min_clusters, max_clusters + 1), inertia_values, marker='o', linestyle='-')
plt.title('Elbow Method for Optimal K')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.xticks(range(min_clusters, max_clusters + 1))
plt.grid(True)
plt.show()

print("Inertia values calculated and plotted.")

In [ ]:
from sklearn.cluster import KMeans

# 1. Initialize a KMeans object with n_clusters=10 and random_state=42
k = 10 # Assuming 10 is chosen as the optimal number of clusters from the elbow plot
kmeans_model = KMeans(n_clusters=k, random_state=42, n_init=10)

# 2. Fit the KMeans model to the features_df_encoded DataFrame
kmeans_model.fit(features_df_encoded)

# 3. Predict the cluster labels for features_df_encoded
cluster_labels = kmeans_model.predict(features_df_encoded)

# 4. Add these cluster labels as a new column named 'Cluster_Label' to the merged_places_df DataFrame
merged_places_df['Cluster_Label'] = cluster_labels

print(f"K-Means clustering applied with {k} clusters.")
print("First 5 rows of merged_places_df with new 'Cluster_Label' column:")
print(merged_places_df.head())

In [ ]:
print("Here's a summary of the models trained in the notebook:")

print("\n1. **LightGBM Ranker (`lgb_ranker_model`)**:")
print("   - **Purpose**: To rank tourist places based on visitor footfall and other features.")
print("   - **Features Used**: `Cost_INR`, `Avg_Temperature_C`, `Rainfall_mm`, `Humidity`.")
print("   - **Target**: Scaled `Total_Visitors` to represent relevance for ranking.")
print("   - **Evaluation**: Achieved a Mean NDCG Score of 0.8155 on the test set, indicating good ranking performance.")

print("\n2. **K-Means Clustering (`kmeans_model`)**:")
print("   - **Purpose**: To group similar tourist places into clusters based on their characteristics.")
print("   - **Features Used**: `Latitude`, `Longitude`, `Total_Visitors`, `Category`, `Sub_Category`, and `Tags` (after preprocessing).")
print("   - **Number of Clusters (k)**: 10 (determined from the Elbow Method visualization).")
print("   - **Output**: Each place in `merged_places_df` is assigned a `Cluster_Label`.")

print("\n3. **XGBoost Regressor (`xgb_regressor_model`)**:")
print("   - **Purpose**: To predict accommodation ratings.")
print("   - **Features Used**: `Rating`, `Latitude`, `Longitude`, one-hot encoded `Type`, `Price_Range`, and `Amenities`.")
print("   - **Target**: `Rating` of accommodations.")
print("   - **Evaluation**: Achieved a Mean Squared Error (MSE) of 0.0000 and an R-squared (R2) Score of 1.0000 on the test set. This indicates a highly accurate model for predicting accommodation ratings.")

In [ ]:
df_footfall_agg = df1.groupby('Place_Name')['Total_Visitors'].sum().reset_index()
df_footfall_agg.head()

In [ ]:
df2=pd.read_csv("/content/drive/MyDrive/Maharashtra datasets/Travel_cost.csv")
df2.head()

In [ ]:
df_costs = df2.groupby(['Place_Name', 'District'])['Cost_INR'].mean().reset_index()
df_costs.head()

In [ ]:
merged_places_df = pd.merge(df5, df_footfall_agg, on='Place_Name', how='left')
print("merged_places_df Head:")
print(merged_places_df.head())
print("\nmerged_places_df Info:")
merged_places_df.info()

In [ ]:
print(f"Number of missing 'Total_Visitors' before imputation: {merged_places_df['Total_Visitors'].isnull().sum()}")
merged_places_df['Total_Visitors'].fillna(merged_places_df['Total_Visitors'].mean(), inplace=True)
print(f"Number of missing 'Total_Visitors' after imputation: {merged_places_df['Total_Visitors'].isnull().sum()}")

In [ ]:
print(f"Number of missing 'Total_Visitors' before imputation: {merged_places_df['Total_Visitors'].isnull().sum()}")
merged_places_df['Total_Visitors'] = merged_places_df['Total_Visitors'].fillna(merged_places_df['Total_Visitors'].mean())
print(f"Number of missing 'Total_Visitors' after imputation: {merged_places_df['Total_Visitors'].isnull().sum()}")

In [ ]:
places_df_processed = merged_places_df.copy()
places_df_processed = pd.merge(places_df_processed, df_costs, on=['Place_Name', 'District'], how='left')

places_df_processed.rename(columns={
    'Place_ID': 'ID',
    'Place_Name': 'Name',
    'Total_Visitors': 'Value',
    'Cost_INR': 'Cost'
}, inplace=True)
places_df_processed['Type'] = 'Place'

places_df_final = places_df_processed[['ID', 'Name', 'District', 'Type', 'Value', 'Cost']]
places_df_final['Cost'].fillna(places_df_final['Cost'].mean(), inplace=True)

print("First 5 rows of places_df_final:")
print(places_df_final.head())
print("\nInfo of places_df_final:")
places_df_final.info()

In [ ]:
places_df_processed = merged_places_df.copy()
places_df_processed = pd.merge(places_df_processed, df_costs, on=['Place_Name', 'District'], how='left')

places_df_processed.rename(columns={
    'Place_ID': 'ID',
    'Place_Name': 'Name',
    'Total_Visitors': 'Value',
    'Cost_INR': 'Cost'
}, inplace=True)
places_df_processed['Type'] = 'Place'

places_df_final = places_df_processed[['ID', 'Name', 'District', 'Type', 'Value', 'Cost']].copy()
places_df_final['Cost'] = places_df_final['Cost'].fillna(places_df_final['Cost'].mean())

print("First 5 rows of places_df_final:")
print(places_df_final.head())
print("\nInfo of places_df_final:")
places_df_final.info()